In [5]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.layers import Activation, Flatten, Bidirectional
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.metrics import roc_auc_score,confusion_matrix, accuracy_score, make_scorer, f1_score,precision_score,recall_score, plot_confusion_matrix
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('rslp')
nltk.download('stopwords')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\cfpc2\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cfpc2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
dataset = pd.read_csv("datasets/reviews.csv")

In [7]:
dataset.head(5)

,Unnamed: 0,order_id,review_id,review_score,review_comment_message
0,3,658677c97b385a9be170737859d3511b,e64fb393e7b32834bb789ff8bb30750e,1,Recebi bem antes do prazo estipulado.
1,4,8e6bfb81e283fa7e4f11123a3fb894f1,f7c4243c7fe1938f181bec41a392bdeb,1,Parabéns lojas lannister adorei comprar pela I...
2,9,b9bf720beb4ab3728760088589c62129,8670d52e15e00043ae7de4c01cc2fe06,0,aparelho eficiente. no site a marca do aparelh...
3,12,9d6f15f95d01e79bd1349cc208361f09,4b49719c8a200003f700d3d986ea1a19,0,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,15,e51478e7e277a83743b6f9991dbfa3fb,3948b09f7c818e2d86c9a546758b2335,1,"Vendedor confiável, produto ok e entrega antes..."


In [20]:
stopword = stopwords.words("portuguese")
stem = RSLPStemmer()
vectorizer = CountVectorizer()
def clear(review):
  review = review.lower()
  # remove pula de linha 
  review = re.sub('\n', ' ', review)        
  review = re.sub('\r', ' ', review)

  # remove numero 
  review = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' #numero ', review)

  # remove caracters especiais 
  review = re.sub(r'R\$', ' ', review)
  review = re.sub(r'\W', ' ', review)
  review = re.sub(r'\s+', ' ', review)

  # remove links 
  urls = re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', review)
  if len(urls) > 0:
      for url in urls:
          for link in url:
              review = review.replace(link, '')
      review = review.replace(':', '')
      review = review.replace('/', '')
  return review
reviews = dataset["review_comment_message"]
reviews = reviews.apply(lambda review: clear(review))
reviews = reviews.apply(lambda review: word_tokenize(review))
reviews = reviews.apply(lambda words_review: [word for word in words_review if word not in stopword])
reviews = reviews.apply(lambda words_review: [stem.stem(word) for word in words_review ])
reviews = reviews.apply(lambda words_review: " ".join(words_review))
# print(reviews)
reviews = vectorizer.fit_transform(reviews)
reviews


<40874x7920 sparse matrix of type '<class 'numpy.int64'>'
	with 277461 stored elements in Compressed Sparse Row format>

In [30]:
print(reviews)

  (0, 6277)	1
  (0, 852)	1
  (0, 431)	1
  (0, 5819)	1
  (0, 2903)	1
  (1, 5409)	2
  (1, 4377)	1
  (1, 4221)	1
  (1, 149)	1
  (1, 1540)	1
  (1, 3979)	1
  (1, 6775)	1
  (1, 6041)	1
  (1, 7292)	1
  (1, 3150)	1
  (1, 6092)	1
  (2, 478)	3
  (2, 2483)	1
  (2, 6921)	1
  (2, 4543)	2
  (2, 3775)	1
  (2, 5115)	1
  (2, 2128)	1
  (2, 1324)	1
  (2, 5342)	1
  :	:
  (40871, 5940)	1
  (40871, 6818)	1
  (40871, 1284)	1
  (40871, 2727)	2
  (40871, 228)	1
  (40871, 4473)	1
  (40871, 5693)	1
  (40871, 3185)	1
  (40871, 5023)	3
  (40871, 3009)	1
  (40872, 4377)	1
  (40872, 2697)	1
  (40872, 6318)	1
  (40872, 7067)	2
  (40872, 6634)	1
  (40872, 4762)	1
  (40872, 2982)	1
  (40873, 6775)	1
  (40873, 1324)	1
  (40873, 5940)	1
  (40873, 5702)	1
  (40873, 1958)	1
  (40873, 4063)	1
  (40873, 2206)	1
  (40873, 1176)	1


In [31]:
X = reviews.copy()
y = dataset["review_score"].copy()
y = np.array(y)

In [32]:
for i in range(0,len(y)):
    if y[i] == -1:
        y[i] = 2
print(y)

[1 1 0 ... 0 1 2]


In [33]:
y_dummy = np_utils.to_categorical(y)
print(y_dummy)
print(y)

[[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[1 1 0 ... 0 1 2]


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 199)

In [35]:
X_train[0]

<1x7920 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [36]:
top_words = 194463
max_review_length = 400
embedding_vecor_length = 50
max_features =194463
embedding_dim =16
sequence_length = 100


In [37]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# model.add(Dropout(0.2))
model.add(LSTM(100))
# model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 400, 50)           9723150   
                                                                 
 lstm_2 (LSTM)               (None, 100)               60400     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 9,783,853
Trainable params: 9,783,853
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
result = model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1)

InvalidArgumentError: indices[1] = [0,5940] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
print(classification_report(y_test, predicted_y_transform))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      3208
           0       0.24      1.00      0.38      2918
           1       0.00      0.00      0.00      6137

    accuracy                           0.24     12263
   macro avg       0.08      0.33      0.13     12263
weighted avg       0.06      0.24      0.09     12263



C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
